In [61]:
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn

torch.set_printoptions(edgeitems=2, linewidth=75)

In [62]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # <1>
t_u = torch.tensor(t_u).unsqueeze(1) # <1>

t_u.shape

torch.Size([11, 1])

In [63]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([ 9,  5,  6,  3,  1,  8,  4, 10,  0]), tensor([2, 7]))

In [64]:
t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

In [65]:
linear_model = nn.Linear(1, 1)
linear_model(t_un_val)

tensor([[-4.2264],
        [-2.1908]], grad_fn=<AddmmBackward0>)

In [66]:
linear_model.weight

Parameter containing:
tensor([[-0.5592]], requires_grad=True)

In [67]:
linear_model.bias

Parameter containing:
tensor([-0.9717], requires_grad=True)

In [68]:
x = torch.ones(1)
linear_model(x)

tensor([-1.5310], grad_fn=<AddBackward0>)

In [69]:
x = torch.ones(10, 1)
linear_model(x)

tensor([[-1.5310],
        [-1.5310],
        [-1.5310],
        [-1.5310],
        [-1.5310],
        [-1.5310],
        [-1.5310],
        [-1.5310],
        [-1.5310],
        [-1.5310]], grad_fn=<AddmmBackward0>)

In [70]:
#hand rolled loss function
def loss_fn(t_p, t_c):
    '''Loss function that we will use for testing purposes'''
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

def training_loop(n_epochs, optimizer, model,
                 loss_fn, t_u_train, t_u_val,
                 t_c_train, t_c_val):
    '''Training loop function for PyTorch'''
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)
        
        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        
        if (epoch == 1) or (epoch % 1000) == 0:
            print(f'Epoch {epoch}, Training loss {loss_train.item():.4f}',
                 f'Validation loss {loss_val.item():.4f}')

In [71]:
linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(
    linear_model.parameters(),
    lr=1e-2)



training_loop(
    n_epochs=3000,
    optimizer=optimizer,
    model=linear_model,
    loss_fn=loss_fn,
    t_u_train=t_un_train,
    t_u_val=t_un_val,
    t_c_train=t_c_train,
    t_c_val=t_c_val
)

Epoch 1, Training loss 133.4373 Validation loss 85.3089
Epoch 1000, Training loss 4.9529 Validation loss 1.0864
Epoch 2000, Training loss 3.1032 Validation loss 2.3444
Epoch 3000, Training loss 2.9510 Validation loss 3.8533


In [72]:
linear_model.parameters()

<generator object Module.parameters at 0x7fc49823ac10>

In [73]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[5.5522]], requires_grad=True),
 Parameter containing:
 tensor([-18.5765], requires_grad=True)]

In [74]:
def training_loop(n_epochs, optimizer, model,
                 loss_fn, t_u_train, t_u_val,
                 t_c_train, t_c_val):
    '''Training loop function for PyTorch'''
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)
        
        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        
        if (epoch == 1) or (epoch % 1000) == 0:
            print(f'Epoch {epoch}, Training loss {loss_train.item():.4f}',
                 f'Validation loss {loss_val.item():.4f}')

In [75]:
linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
    n_epochs=3000,
    optimizer=optimizer,
    model=linear_model,
    loss_fn=nn.MSELoss(),
    t_u_train=t_un_train,
    t_u_val=t_un_val,
    t_c_train=t_c_train,
    t_c_val=t_c_val
)

print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 249.6921 Validation loss 146.0606
Epoch 1000, Training loss 5.0328 Validation loss 1.1397
Epoch 2000, Training loss 3.1098 Validation loss 2.3101
Epoch 3000, Training loss 2.9516 Validation loss 3.8394

Parameter containing:
tensor([[5.5506]], requires_grad=True)
Parameter containing:
tensor([-18.5674], requires_grad=True)


In [77]:
seq_model = nn.Sequential(
    nn.Linear(1,13),
    nn.Tanh(),
    nn.Linear(3, 1))

seq_model

Sequential(
  (0): Linear(in_features=1, out_features=13, bias=True)
  (1): Tanh()
  (2): Linear(in_features=3, out_features=1, bias=True)
)

In [79]:
[param.shape for param in seq_model.parameters()]

[torch.Size([13, 1]), torch.Size([13]), torch.Size([1, 3]), torch.Size([1])]